# 008: Notebook to deploy Power BI Project with PyFabricOps 

In [76]:
# Libraries
import json
import os
import pyfabricops as pf

# Enable logging and set authentication provider
pf.enable_notebook_logging() # Enable logging for notebook
pf.set_auth_provider('env') 

# Get repo root - works in Jupyter notebooks
repo_root = os.path.abspath('..')  # Goes up one level from notebooks folder
project = 'PowerBIDemo'
project_path = os.path.join(repo_root, 'src', project)
print(f"Project path: {project_path}")

# Define branches configuration path
branches_path = os.path.join(repo_root, 'branches.json')
print(f"Branches configuration path: {branches_path}")

# Load branches configuration
branch = pf.get_current_branch()
workspace_suffix = pf.get_workspace_suffix(path=branches_path)

# Define workspace name based on project and branch
workspace_name = project + workspace_suffix
print(f'Branch: {branch}, Workspace: {workspace_name}') 

Project path: c:\repos\pyfabricops-examples\src\PowerBIDemo
Branches configuration path: c:\repos\pyfabricops-examples\branches.json
Branch: main, Workspace: PowerBIDemo-PRD


In [77]:
# Retrienving the workspace_id from config.json in the project_path for better performance
config_path = f'{project_path}/config.json'
with open(config_path, 'r') as f:
  config_content = json.load(f)
  config = config_content[branch]  
  workspace_id = config[project]['workspace_config']['workspace_id'] 
print(f'Workspace ID: {workspace_id}')

Workspace ID: aeccbc0a-6e35-4c65-b29b-f2f5be3689bf


In [78]:
# Deploy the folders first.
pf.deploy_folders(
	workspace=workspace_id, 
	project_path=project_path,
	branches_path=branches_path,
)  

pyfabricops._folders INFO: Found 2 folders containing Fabric artifacts
pyfabricops._folders WARNING: 409: {"requestId":"e219eae2-560d-4dce-b7b2-498c01246448","errorCode":"FolderDisplayNameAlreadyInUse","message":"Folder display name is already used."}.
pyfabricops._folders WARNING: 409: {"requestId":"e219eae2-560d-4dce-b7b2-498c01246448","errorCode":"FolderDisplayNameAlreadyInUse","message":"Folder display name is already used."}.
pyfabricops._folders WARNING: 409: {"requestId":"eb9f3da1-6ceb-4168-8748-8061c16ee33d","errorCode":"FolderDisplayNameAlreadyInUse","message":"Folder display name is already used."}.
pyfabricops._folders INFO: Created folders for workspace aeccbc0a-6e35-4c65-b29b-f2f5be3689bf.
pyfabricops._folders WARNING: 409: {"requestId":"eb9f3da1-6ceb-4168-8748-8061c16ee33d","errorCode":"FolderDisplayNameAlreadyInUse","message":"Folder display name is already used."}.
pyfabricops._folders INFO: Created folders for workspace aeccbc0a-6e35-4c65-b29b-f2f5be3689bf.
pyfabricops

In [79]:
# Deploy the calendar dataflow gen1
# Due a conflict with others dataflow generations and limitations with folders we need deploy it separately.
pf.deploy_dataflow_gen1(
	workspace=workspace_id, 
	path=f'{project_path}/workspace/Calendar.Dataflow',
)  

pyfabricops._utils INFO: Loaded JSON file with json5: c:\repos\pyfabricops-examples\src\PowerBIDemo/workspace/Calendar.Dataflow\model.json
pyfabricops._dataflows_gen1 ERROR: Error deploying the dataflow: 400 - {'code': 'Conflict', 'message': "Dataflow 'Calendar' already exists"}
pyfabricops._dataflows_gen1 ERROR: Error deploying the dataflow: 400 - {'code': 'Conflict', 'message': "Dataflow 'Calendar' already exists"}


In [80]:
# Export dataflow config
# Dataflows gen1 don't have folders support on export.
# They arrive in the root of the workspace.
pf.export_all_dataflows_gen1(
	workspace=workspace_id,
				project_path=project_path,
	branches_path=branches_path,
)

pyfabricops._dataflows_gen1 INFO: Exported dataflow Calendar to c:\repos\pyfabricops-examples\src\PowerBIDemo\workspace\Calendar.Dataflow.
pyfabricops._dataflows_gen1 INFO: Found existing config file at c:\repos\pyfabricops-examples\src\PowerBIDemo\config.json, merging workspace config...
pyfabricops._dataflows_gen1 INFO: Found existing config file at c:\repos\pyfabricops-examples\src\PowerBIDemo\config.json, merging workspace config...


> [!IMPORTANT]
> If is a first time you need create a minimal config.json file for the semantic models.

For example:
```json
{
	"dev": {
		"PowerBIDemo": {
			"semantic_models": {
				"Sales": {
					"parameters": {
						"Server": {"Value": "PEZZOTT"},
						"Database": {"Value": "AdventureWorksDW2022-DEV"},
						"WorkspaceId": {"Value": "de8f92a3-5b98-461a-b27d-c8a1c60c0cfc"}
					}
				}
			}
		}
	}
}
``` 


In [81]:
# Read the config.json retriving the parameters for the semantic model
with open(config_path, 'r') as f:
	config_content = json.load(f)
config = config_content[branch]

semantic_models_config = config[project]['semantic_models']

In [82]:
# Replacing the parameters in the semantic models with the values from config.json
import glob 
import re

for semantic_model_path in glob.glob(f'{project_path}/**/*.SemanticModel', recursive=True):
	print(f'Processing semantic model: {semantic_model_path}')
	semantic_model_name = semantic_model_path.replace('\\', '/').split('/')[-1].split('.SemanticModel')[0] 
	print(f'Deploying semantic model name: {semantic_model_name}')  

	# Retrieving the semantic model config
	semantic_model_config = semantic_models_config[semantic_model_name]
	semantic_model_parameters = semantic_model_config['parameters']

	# Read the expressions file
	expressions_path = f'{semantic_model_path}//definition/expressions.tmdl'
	with open(expressions_path, 'r') as f:
		expressions = f.read()

	# Build variables dictionary dynamically from config.json
	variables_to_replace = {}
	for parameter_name, parameter_obj in semantic_model_parameters.items():
		variables_to_replace[parameter_name] = parameter_obj['Value']
		print(f"Found parameter: {parameter_name} = {parameter_obj['Value']}")
	
	# Start with original expressions
	expressions_updated = expressions

	# Replace each variable using regex
	for variable_name, new_value in variables_to_replace.items():
		# Pattern to match: VariableName = "any_value"
		pattern = rf'{variable_name}\s*=\s*"[^"]*"'
		replacement = f'{variable_name} = "{new_value}"'
		expressions_updated = re.sub(pattern, replacement, expressions_updated)
		print(f"Replaced {variable_name} with: {new_value}")

	# Write the updated expressions back to the file
	with open(expressions_path, 'w', encoding='utf-8') as f:
		f.write(expressions_updated)

Processing semantic model: c:\repos\pyfabricops-examples\src\PowerBIDemo\workspace\Main\Sales.SemanticModel
Deploying semantic model name: Sales
Found parameter: Server = PEZZOTT
Found parameter: Database = AdventureWorksDW2022-DEV
Found parameter: WorkspaceId = aeccbc0a-6e35-4c65-b29b-f2f5be3689bf
Replaced Server with: PEZZOTT
Replaced Database with: AdventureWorksDW2022-DEV
Replaced WorkspaceId with: aeccbc0a-6e35-4c65-b29b-f2f5be3689bf


In [83]:
# Deploy all the semantic models
pf.deploy_all_semantic_models(
	workspace=workspace_id, 
	project_path=project_path,
	branches_path=branches_path,
)

pyfabricops._semantic_models INFO: Semantic model 'Sales' already exists, updating...
pyfabricops._semantic_models INFO: Successfully updated semantic model 'Sales'
pyfabricops._semantic_models INFO: Successfully updated semantic model 'Sales'
pyfabricops._semantic_models INFO: Deployment completed. Successfully deployed 1 semantic models.
pyfabricops._semantic_models INFO: Deployment completed. Successfully deployed 1 semantic models.


['Sales']

In [84]:
# Retrieving the semantic_model_details
pf.export_all_semantic_models(
	workspace=workspace_id, 
	project_path=project_path,
	branches_path=branches_path,
)

pyfabricops._semantic_models INFO: Found existing config file at c:\repos\pyfabricops-examples\src\PowerBIDemo\config.json, merging workspace config...
pyfabricops._utils INFO: Item definition unpacked to c:\repos\pyfabricops-examples\src\PowerBIDemo\workspace\Main/Sales.SemanticModel
pyfabricops._utils INFO: Item definition unpacked to c:\repos\pyfabricops-examples\src\PowerBIDemo\workspace\Main/Sales.SemanticModel


In [85]:
# After deploying the semantic models, we change the values to placeholders to commit the changes.
import glob
import re

for semantic_model_path in glob.glob(f'{project_path}/**/*.SemanticModel', recursive=True):
	print(f'Processing semantic model: {semantic_model_path}')
	semantic_model_name = semantic_model_path.replace('\\', '/').split('/')[-1].split('.SemanticModel')[0] 
	print(f'Injecting placeholders into: {semantic_model_name}')  

	# Retrieve the parameters to build the placeholders
	semantic_model_config = semantic_models_config[semantic_model_name]
	semantic_model_parameters = semantic_model_config['parameters']

	# Read the current content of expressions.tmdl (with the actual values)
	expressions_path = f'{semantic_model_path}/definition/expressions.tmdl'
	with open(expressions_path, 'r', encoding='utf-8') as f:
		expressions = f.read()

	# Replace the values with placeholders
	expressions_with_placeholders = expressions

	for parameter_name, parameter_obj in semantic_model_parameters.items():
		actual_value = parameter_obj["Value"]
		# Create the pattern to replace: Param = "CurrentValue"
		pattern = rf'{parameter_name}\s*=\s*"{re.escape(actual_value)}"'
		# Replace: Param = "#{Param}#"
		replacement = f'{parameter_name} = "#{{{parameter_name}}}#"'
		expressions_with_placeholders = re.sub(pattern, replacement, expressions_with_placeholders)
		print(f"Replaced {parameter_name} value with placeholder.")

	# Write back the result on file
	with open(expressions_path, 'w', encoding='utf-8') as f:
		f.write(expressions_with_placeholders)




Processing semantic model: c:\repos\pyfabricops-examples\src\PowerBIDemo\workspace\Main\Sales.SemanticModel
Injecting placeholders into: Sales
Replaced Server value with placeholder.
Replaced Database value with placeholder.
Replaced WorkspaceId value with placeholder.


In [86]:
# For each report, we will define the report attached to the semantic model and deploy it.
import glob 
import re

for report_path in glob.glob(f'{project_path}/**/*.Report', recursive=True):
	print(f'Processing report: {report_path}')
	report_name = report_path.replace('\\', '/').split('/')[-1].split('.Report')[0] 
	print(f'Deploying report: {report_name}')

	with open(f'{report_path}/definition.pbir', 'r') as f:
		report_definition = json.load(f)

	dataset_reference = report_definition['datasetReference']

	if 'byPath' in dataset_reference:
		dataset_path = dataset_reference['byPath']['path'] 
		dataset_name = dataset_path.split('/')[-1].split('.SemanticModel')[0]

	elif 'byConnection' in dataset_reference:
		text_to_search = dataset_reference['byConnection']['connectionString']
		# Capture the value after "initial catalog="
		match = re.search(r'initial catalog=([^;]+)', text_to_search)
		if match:
			dataset_name = match.group(1)

	print(f'Semantic model: {dataset_name}')
	
	# Search for the semantic model in the config.json
	with open(config_path, 'r') as f:
		config_content = json.load(f)
	config = config_content[branch]
	semantic_model_id = config[project]['semantic_models'][dataset_name]['id'] 
	print(f'Semantic Model ID: {semantic_model_id}')  
	
	# Replace the definition.pbir with the updated template
	with open(os.path.join(repo_root, 'template_report_definition.pbir'), 'r', encoding='utf-8') as f:
		report_definition_template = f.read()

	report_definition_updated = report_definition_template.replace('{workspace_name}', workspace_name)
	report_definition_updated = report_definition_updated.replace('{semantic_model_name}', dataset_name)
	report_definition_updated = report_definition_updated.replace('{semantic_model_id}', semantic_model_id)
	# print(report_definition_updated)    

	with open(f'{report_path}/definition.pbir', 'w', encoding='utf-8') as f:
		f.write(report_definition_updated)
	
	# Deploy the report
	pf.deploy_report(
		workspace=workspace_id, 
		display_name=report_name,
		project_path=project_path,
		branches_path=branches_path,
	)
	
	# Write back the original report_definition for the definition.pbir
	with open(f'{report_path}/definition.pbir', 'w', encoding='utf-8') as f:
		json.dump(report_definition, f, indent=2)


Processing report: c:\repos\pyfabricops-examples\src\PowerBIDemo\workspace\Customers\Customers.Report
Deploying report: Customers
Semantic model: Sales
Semantic Model ID: 4b45af69-6b7f-4a56-9e40-676506fc0c61


pyfabricops._reports INFO: Report 'Customers' already exists, updating...
pyfabricops._reports INFO: Successfully updated report 'Customers'


Processing report: c:\repos\pyfabricops-examples\src\PowerBIDemo\workspace\Main\Sales.Report
Deploying report: Sales
Semantic model: Sales
Semantic Model ID: 4b45af69-6b7f-4a56-9e40-676506fc0c61


pyfabricops._reports INFO: Report 'Sales' already exists, updating...
pyfabricops._reports INFO: Successfully updated report 'Sales'
